In [1]:
# CELL 1: Imports, paths, and load existing team_game_logs

from pathlib import Path
import time
import numpy as np
import pandas as pd
from nba_api.stats.endpoints import TeamGameLog

# ---- Project + raw data paths ----
project_root = Path.cwd().resolve()
raw_root = project_root / "data" / "raw"
raw_root.mkdir(parents=True, exist_ok=True)

team_logs_path = raw_root / "team_game_logs.csv"

print("Project root:", project_root)
print("Raw data root:", raw_root)
print("team_game_logs path:", team_logs_path)

if not team_logs_path.exists():
    raise FileNotFoundError(
        f"{team_logs_path} does not exist. "
        "Run your build_team_game_logs notebook first to create it."
    )

# ---- Load existing logs ----
existing = pd.read_csv(team_logs_path, parse_dates=["gameDate"])

print("\n=== Existing team_game_logs summary ===")
print("Shape:", existing.shape)
print("Date range:", existing["gameDate"].min(), "→", existing["gameDate"].max())
print("Seasons present:", sorted(existing["Season"].unique()))

# Latest season in the file (string like "2025-26")
latest_season = sorted(existing["Season"].unique())[-1]
latest_date = existing["gameDate"].max()

print(f"\nLatest season in file: {latest_season}")
print(f"Latest gameDate in file: {latest_date}")

# ---- CONFIG: which seasons to update ----
# By default we only update the *latest* season, which is what you want
# for a continually growing database.
FORCE_SEASONS = None  # e.g. ["2024-25", "2025-26"] if you ever want to override

if FORCE_SEASONS is None:
    seasons_to_update = [latest_season]
else:
    seasons_to_update = FORCE_SEASONS

print("\nSeasons to update:", seasons_to_update)

# ---- Team ID → (abbrev, name) map (same as build notebook) ----
TEAM_ID_TO_META = {
    1610612737: ("ATL", "Atlanta Hawks"),
    1610612738: ("BOS", "Boston Celtics"),
    1610612739: ("CLE", "Cleveland Cavaliers"),
    1610612740: ("NOP", "New Orleans Pelicans"),
    1610612741: ("CHI", "Chicago Bulls"),
    1610612742: ("DAL", "Dallas Mavericks"),
    1610612743: ("DEN", "Denver Nuggets"),
    1610612744: ("GSW", "Golden State Warriors"),
    1610612745: ("HOU", "Houston Rockets"),
    1610612746: ("LAC", "LA Clippers"),
    1610612747: ("LAL", "Los Angeles Lakers"),
    1610612748: ("MIA", "Miami Heat"),
    1610612749: ("MIL", "Milwaukee Bucks"),
    1610612750: ("MIN", "Minnesota Timberwolves"),
    1610612751: ("BKN", "Brooklyn Nets"),
    1610612752: ("NYK", "New York Knicks"),
    1610612753: ("ORL", "Orlando Magic"),
    1610612754: ("IND", "Indiana Pacers"),
    1610612755: ("PHI", "Philadelphia 76ers"),
    1610612756: ("PHX", "Phoenix Suns"),
    1610612757: ("POR", "Portland Trail Blazers"),
    1610612758: ("SAC", "Sacramento Kings"),
    1610612759: ("SAS", "San Antonio Spurs"),
    1610612760: ("OKC", "Oklahoma City Thunder"),
    1610612761: ("TOR", "Toronto Raptors"),
    1610612762: ("UTA", "Utah Jazz"),
    1610612763: ("MEM", "Memphis Grizzlies"),
    1610612764: ("WAS", "Washington Wizards"),
    1610612765: ("DET", "Detroit Pistons"),
    1610612766: ("CHA", "Charlotte Hornets"),
}

abbrev_to_id = {abbrev: tid for tid, (abbrev, _) in TEAM_ID_TO_META.items()}
abbrev_to_name = {abbrev: name for _, (abbrev, name) in TEAM_ID_TO_META.items()}

# Canonical schema (same as build notebook)
TEAM_LOG_SCHEMA = [
    "Season",
    "gameId",
    "gameDate",
    "teamId",
    "teamAbbrev",
    "teamName",
    "opponentTeamId",
    "opponentTeamAbbrev",
    "opponentTeamName",
    "home",
    "win",
    "minutes",
    "pts",
    "reboundsTotal",
    "assists",
    "steals",
    "blocks",
    "turnovers",
    "foulsPersonal",
    "fieldGoalsMade",
    "fieldGoalsAttempted",
    "fieldGoalsPercentage",
    "threePointersMade",
    "threePointersAttempted",
    "threePointersPercentage",
    "freeThrowsMade",
    "freeThrowsAttempted",
    "freeThrowsPercentage",
    "reboundsOffensive",
    "reboundsDefensive",
]

print("\nTEAM_LOG_SCHEMA columns:", len(TEAM_LOG_SCHEMA))


Project root: C:\Users\wdors\qepc_project\experimental\GTP_REWRITE\qepc_core\notebooks
Raw data root: C:\Users\wdors\qepc_project\experimental\GTP_REWRITE\qepc_core\notebooks\data\raw
team_game_logs path: C:\Users\wdors\qepc_project\experimental\GTP_REWRITE\qepc_core\notebooks\data\raw\team_game_logs.csv

=== Existing team_game_logs summary ===
Shape: (10456, 30)
Date range: 2021-10-19 00:00:00 → 2025-12-01 00:00:00
Seasons present: ['2021-22', '2022-23', '2023-24', '2024-25', '2025-26']

Latest season in file: 2025-26
Latest gameDate in file: 2025-12-01 00:00:00

Seasons to update: ['2025-26']

TEAM_LOG_SCHEMA columns: 30


In [2]:
# CELL 2: normalize_teamgamelog_df (same logic as build notebook)

def normalize_teamgamelog_df(df_raw: pd.DataFrame, season: str) -> pd.DataFrame:
    """
    Take the raw df from nba_api TeamGameLog and convert into TEAM_LOG_SCHEMA.
    Defensive about column name casing and variants.
    """
    df = df_raw.copy()
    df["Season"] = season

    cols_lower = {c.lower(): c for c in df.columns}

    def get_exact_ci(*candidates):
        for cand in candidates:
            real = cols_lower.get(cand.lower())
            if real is not None:
                return real
        return None

    def find_by_substrings(*substrings):
        wanted = [s.lower() for s in substrings]
        for c in df.columns:
            cl = c.lower()
            if all(s in cl for s in wanted):
                return c
        return None

    # IDs
    game_id_col = get_exact_ci("GAME_ID", "Game_ID") or find_by_substrings("game", "id")
    team_id_col = get_exact_ci("TEAM_ID", "Team_ID") or find_by_substrings("team", "id")
    team_abbrev_col = (
        get_exact_ci("TEAM_ABBREVIATION", "Team_Abbreviation")
        or find_by_substrings("team", "abbrev")
    )
    team_name_col = (
        get_exact_ci("TEAM_NAME", "Team_Name")
        or find_by_substrings("team", "name")
    )
    game_date_col = get_exact_ci("GAME_DATE", "Game_Date") or find_by_substrings("game", "date")
    matchup_col = get_exact_ci("MATCHUP") or find_by_substrings("matchup")
    wl_col = get_exact_ci("WL") or find_by_substrings("wl")
    min_col = get_exact_ci("MIN") or find_by_substrings("min")

    df["gameId"] = df[game_id_col] if game_id_col else pd.NA
    df["teamId"] = df[team_id_col] if team_id_col else pd.NA
    df["teamAbbrev"] = df[team_abbrev_col] if team_abbrev_col else pd.NA
    df["teamName"] = df[team_name_col] if team_name_col else pd.NA

    if game_date_col:
        df["gameDate"] = pd.to_datetime(
            df[game_date_col].astype(str),
            format="%b %d, %Y",
            errors="coerce",
        )
    else:
        df["gameDate"] = pd.NaT

    if wl_col:
        df["win"] = (df[wl_col] == "W").astype(int)
    else:
        df["win"] = pd.NA

    df["minutes"] = df[min_col] if min_col else pd.NA

    # matchup → home/away + opponent abbrev
    if matchup_col:
        matchup = df[matchup_col].astype(str)
        df["home"] = matchup.str.contains(" vs\.").astype(int)
        opp_abbrev = matchup.str.split().str[-1]
        df["opponentTeamAbbrev"] = opp_abbrev
        df["opponentTeamId"] = df["opponentTeamAbbrev"].map(abbrev_to_id)
        df["opponentTeamName"] = df["opponentTeamAbbrev"].map(abbrev_to_name)
    else:
        df["home"] = pd.NA
        df["opponentTeamAbbrev"] = pd.NA
        df["opponentTeamId"] = pd.NA
        df["opponentTeamName"] = pd.NA

    # stats
    def copy_stat(src_candidates, dst):
        src_col = get_exact_ci(*src_candidates)
        if not src_col:
            src_col = find_by_substrings(*src_candidates)
        if src_col:
            df[dst] = df[src_col]
        else:
            df[dst] = pd.NA

    copy_stat(["PTS"], "pts")
    copy_stat(["REB"], "reboundsTotal")
    copy_stat(["AST"], "assists")
    copy_stat(["STL"], "steals")
    copy_stat(["BLK"], "blocks")
    copy_stat(["TOV"], "turnovers")
    copy_stat(["PF"], "foulsPersonal")
    copy_stat(["PLUS_MINUS", "PLUSMINUS"], "plusMinusPoints")

    copy_stat(["FGM"], "fieldGoalsMade")
    copy_stat(["FGA"], "fieldGoalsAttempted")
    copy_stat(["FG_PCT", "FG_Pct"], "fieldGoalsPercentage")

    copy_stat(["FG3M"], "threePointersMade")
    copy_stat(["FG3A"], "threePointersAttempted")
    copy_stat(["FG3_PCT", "FG3_Pct"], "threePointersPercentage")

    copy_stat(["FTM"], "freeThrowsMade")
    copy_stat(["FTA"], "freeThrowsAttempted")
    copy_stat(["FT_PCT", "FT_Pct"], "freeThrowsPercentage")

    copy_stat(["OREB"], "reboundsOffensive")
    copy_stat(["DREB"], "reboundsDefensive")

    for col in TEAM_LOG_SCHEMA:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[TEAM_LOG_SCHEMA].copy()
    return df

print("normalize_teamgamelog_df defined.")


normalize_teamgamelog_df defined.


In [3]:
# CELL 3: Helper to patch teamAbbrev/teamName from teamId

def safe_lookup_team_meta(team_id_val):
    if pd.isna(team_id_val):
        return (None, None)
    try:
        tid = int(team_id_val)
    except (ValueError, TypeError):
        return (None, None)
    return TEAM_ID_TO_META.get(tid, (None, None))

def patch_team_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if "teamId" in df.columns:
        abbrevs = []
        names = []
        for v in df["teamId"]:
            ab, nm = safe_lookup_team_meta(v)
            abbrevs.append(ab)
            names.append(nm)

        df["teamAbbrev"] = df.get("teamAbbrev", pd.Series([np.nan]*len(df)))
        df["teamName"] = df.get("teamName", pd.Series([np.nan]*len(df)))

        df["teamAbbrev"] = df["teamAbbrev"].fillna(pd.Series(abbrevs, index=df.index))
        df["teamName"]   = df["teamName"].fillna(pd.Series(names, index=df.index))

    if "opponentTeamId" in df.columns:
        opp_abbrevs = []
        opp_names = []
        for v in df["opponentTeamId"]:
            ab, nm = safe_lookup_team_meta(v)
            opp_abbrevs.append(ab)
            opp_names.append(nm)

        if "opponentTeamAbbrev" not in df.columns:
            df["opponentTeamAbbrev"] = np.nan
        if "opponentTeamName" not in df.columns:
            df["opponentTeamName"] = np.nan

        df["opponentTeamAbbrev"] = df["opponentTeamAbbrev"].fillna(
            pd.Series(opp_abbrevs, index=df.index)
        )
        df["opponentTeamName"] = df["opponentTeamName"].fillna(
            pd.Series(opp_names, index=df.index)
        )

    return df

print("patch_team_names defined.")


patch_team_names defined.


In [4]:
# CELL 4: Fetch new logs for seasons_to_update

team_ids = sorted(TEAM_ID_TO_META.keys())
new_logs = []
errors_update = []

print("=== Updating team_game_logs with new data ===")
print("Seasons to update:", seasons_to_update)
print("Teams:", len(team_ids))

for season in seasons_to_update:
    print(f"\n📅 Season {season}")
    for i, team_id in enumerate(team_ids, start=1):
        abbrev, name = TEAM_ID_TO_META[team_id]
        print(f"  [{i:2d}/{len(team_ids)}] {season} – {abbrev} ({team_id})", end="\r")

        try:
            tgl = TeamGameLog(
                team_id=team_id,
                season=season,
                season_type_all_star="Regular Season",
            )
            df_raw = tgl.get_data_frames()[0]
        except Exception as e:
            msg = f"{season} {team_id} ({abbrev}) – api error: {e}"
            print("\n⚠️", msg)
            errors_update.append(msg)
            time.sleep(1.0)
            continue

        if df_raw.empty:
            msg = f"{season} {team_id} ({abbrev}) – empty frame"
            print("\n⚠️", msg)
            errors_update.append(msg)
            time.sleep(0.5)
            continue

        try:
            df_norm = normalize_teamgamelog_df(df_raw, season=season)
            df_norm = patch_team_names(df_norm)
            new_logs.append(df_norm)
        except Exception as e:
            msg = f"{season} {team_id} ({abbrev}) – normalize/patch error: {e}"
            print("\n⚠️", msg)
            errors_update.append(msg)
            continue

        time.sleep(0.5)

    print(f"\n✅ Finished season {season}")

print("\nFetch for update complete.")
print("New normalized chunks:", len(new_logs))
print("Errors logged:", len(errors_update))


=== Updating team_game_logs with new data ===
Seasons to update: ['2025-26']
Teams: 30

📅 Season 2025-26
  [30/30] 2025-26 – CHA (1610612766)
✅ Finished season 2025-26

Fetch for update complete.
New normalized chunks: 30
Errors logged: 0


In [5]:
# CELL 5: Merge new logs into existing and dedupe

if not new_logs:
    print("⚠️ No new logs fetched; nothing to merge.")
else:
    new_all = pd.concat(new_logs, ignore_index=True)
    print("new_all shape:", new_all.shape)
    print("new_all date range:", new_all["gameDate"].min(), "→", new_all["gameDate"].max())
    print("new_all seasons:", sorted(new_all["Season"].unique()))

    # Combine old + new
    combined = pd.concat([existing, new_all], ignore_index=True)

    # Deduplicate
    key_cols = ["Season", "gameId", "teamId"]
    before = len(combined)
    combined = (
        combined
        .sort_values(key_cols + ["gameDate"])
        .drop_duplicates(subset=key_cols, keep="last")
        .reset_index(drop=True)
    )
    after = len(combined)

    print(f"\nDeduped on {key_cols}: {before} → {after}")
    print("Combined date range:", combined["gameDate"].min(), "→", combined["gameDate"].max())

    # Quick sanity: games per season
    print("\nPer-season game counts (unique gameIds):")
    display(combined.groupby("Season")["gameId"].nunique())

    # Preview some of the newest games
    print("\nNewest 10 rows by gameDate:")
    display(
        combined.sort_values("gameDate", ascending=False)
        .head(10)[
            ["Season", "gameDate", "teamAbbrev", "teamName",
             "opponentTeamAbbrev", "opponentTeamName",
             "home", "win", "pts"]
        ]
    )

    # Keep `combined` for saving in the next cell
    updated_team_game_logs = combined


new_all shape: (628, 30)
new_all date range: 2025-10-21 00:00:00 → 2025-12-02 00:00:00
new_all seasons: ['2025-26']

Deduped on ['Season', 'gameId', 'teamId']: 11084 → 11084
Combined date range: 2021-10-19 00:00:00 → 2025-12-02 00:00:00

Per-season game counts (unique gameIds):


Season
2021-22    1230
2022-23    1230
2023-24    1230
2024-25    1230
2025-26     622
Name: gameId, dtype: int64


Newest 10 rows by gameDate:


,Season,gameDate,teamAbbrev,teamName,opponentTeamAbbrev,opponentTeamName,home,win,pts
11083,2025-26,2025-12-02,OKC,Oklahoma City Thunder,GSW,Golden State Warriors,0,1,124
11077,2025-26,2025-12-02,NYK,New York Knicks,BOS,Boston Celtics,0,0,117
11072,2025-26,2025-12-02,PHI,Philadelphia 76ers,WAS,Washington Wizards,1,1,121
11073,2025-26,2025-12-02,WAS,Washington Wizards,PHI,Philadelphia 76ers,0,0,102
11075,2025-26,2025-12-02,TOR,Toronto Raptors,POR,Portland Trail Blazers,1,1,121
11076,2025-26,2025-12-02,BOS,Boston Celtics,NYK,New York Knicks,1,1,123
11074,2025-26,2025-12-02,POR,Portland Trail Blazers,TOR,Toronto Raptors,0,0,118
11078,2025-26,2025-12-02,NOP,New Orleans Pelicans,MIN,Minnesota Timberwolves,1,0,142
11079,2025-26,2025-12-02,MIN,Minnesota Timberwolves,NOP,New Orleans Pelicans,0,1,149
11080,2025-26,2025-12-02,SAS,San Antonio Spurs,MEM,Memphis Grizzlies,1,1,126


In [6]:
# CELL 6: Save updated team_game_logs back to CSV

if "updated_team_game_logs" not in globals():
    print("⚠️ Nothing to save (no updated_team_game_logs).")
else:
    updated_team_game_logs.to_csv(team_logs_path, index=False)

    print("✅ Updated team_game_logs written to:")
    print("   ", team_logs_path)
    print("Final shape:", updated_team_game_logs.shape)

    print("\nFinal per-season game counts (unique gameIds):")
    display(updated_team_game_logs.groupby("Season")["gameId"].nunique())


✅ Updated team_game_logs written to:
    C:\Users\wdors\qepc_project\experimental\GTP_REWRITE\qepc_core\notebooks\data\raw\team_game_logs.csv
Final shape: (11084, 30)

Final per-season game counts (unique gameIds):


Season
2021-22    1230
2022-23    1230
2023-24    1230
2024-25    1230
2025-26     622
Name: gameId, dtype: int64